In [1]:
import importlib
from itertools import combinations
from typing import List, Tuple, Union

import matplotlib.pyplot as plt
import numpy as np
from numpy.typing import NDArray

import scripts.resection_functions as resection
import scripts.theodolite_function as tfunctions
import scripts.theodolite_utils as tutils


def resection_with_2_known_points(reference_point_1: NDArray, reference_point_2: NDArray, reading_point_1: NDArray, reading_point_2: NDArray) -> Tuple[NDArray, NDArray]:
    """
    Compute the position and orientation of a total station based on two reference points and their respective reading made by the total station.

    The reference points must be in the cartesian coordinate system, i.e. [x, y, z] and the measured points must be in the spherical coordinate system, i.e. [elevation, azimuth, distance].

    Parameters
    ----------
    reference_point_1 : list
        Cartesian coordinates, i.e. [x, y, z], of the first reference point.
    reference_point_2 : list
        Cartesian coordinates, i.e. [x, y, z], of the second reference point.
    reading_point_1 : list
        Spherical coordinates, i.e. [distance, elevation, azimuth], of the first reference point measured by the total station.
    reading_point_2 : list
        Spherical coordinates, i.e. [distance, elevation, azimuth], of the second reference point measured by the total station.

    Returns
    -------
    list[ndarray]
        A list of the two possible position of the total station.
    """
    x_1, y_1, z_1 = reference_point_1
    x_2, y_2, z_2 = reference_point_2
    distance_1, elevation_1, _ = reading_point_1
    distance_2, elevation_2, _ = reading_point_2

    z = np.mean([z_1 - distance_1*np.cos(elevation_1), z_2 - distance_2*np.cos(elevation_2)])

    # find the position of the total station using the intersection of 2 circles given by the measurements of the prism position.
    radius_1 = distance_1*np.sin(elevation_1)
    radius_2 = distance_2*np.sin(elevation_2)

    delta_y = y_2 - y_1

    # find the value of y based on the circles' radius and the reference points
    if delta_y != 0:
        y = (radius_1**2 - radius_2**2 - y_1**2 + y_2**2)/(2*delta_y)
    else:
        y = 0

    # find the possible values of x based on the y value
    x_1_1 = np.sqrt(radius_1**2 - (y - y_1)**2)
    x_1_2 = np.sqrt(radius_2**2 - (y - y_2)**2)
    x_2_1 = -x_1_1
    x_2_2 = -x_1_2

    x_1 = np.mean([x_1_1, x_1_2])
    x_2 = np.mean([x_2_1, x_2_2])

    position_1 = np.array([x_1, y, z, 0, 0, 0])
    position_2 = np.array([x_2, y, z, 0, 0, 0])


    TF1 = tutils.tf_from_pose_roll_pitch_yaw(position_1)
    TF2 = tutils.tf_from_pose_roll_pitch_yaw(position_2)

    reading_1_homogeneous = np.append(spherical_2_cartesian_coords(*reading_point_1), 1)
    reading_2_homogeneous = np.append(spherical_2_cartesian_coords(*reading_point_2), 1)

    R1 = calculate_tf_angle_using_scalar_product(reference_point_1, reference_point_2, TF1@reading_1_homogeneous, TF1@reading_2_homogeneous)
    R2 = calculate_tf_angle_using_scalar_product(reference_point_1, reference_point_2, TF2@reading_1_homogeneous, TF2@reading_2_homogeneous)

    TF1 = TF1@R1
    TF2 = TF2@R2

    return TF1, TF2


def calculate_tf_angle_using_scalar_product(reference_1: NDArray, reference_2: NDArray, reading_1: NDArray, reading_2: NDArray) -> NDArray:
    """
    Compute the angle between two pair of points from two different frames.

    The translation from the reading frame to the reference frame must have been applied prior to compute the angle.

    Parameters
    ----------
    reference_1: ndarray
        The first point's cartesian coordinates in the reference frame.
    reference_2: ndarray
        The second point's cartesian coordinates in the reference frame.
    reading_1: ndarray
        The first point's cartesian coordinates in the reading frame.
    reading_2: ndarray
        The second point's cartesian coordinates in the reading frame.

    Returns
    -------
    ndarray
        Return the angle between two pair of point in two different frames encoded into a 3D rigid transformation matrix.
    """
    ref_line = reference_1[:2] - reference_2[:2]
    ts_line = reading_1[:2] - reading_2[:2]
    angle = np.arccos(np.dot(ref_line, ts_line)/(np.linalg.norm(ts_line)*np.linalg.norm(ref_line)))

    if reading_1[0] > reading_2[0]:
        angle = -angle

    return resection.T_z(angle, [0, 0, 0])


def cartesian_2_spherical_coords(x: float, y: float, z: float) -> NDArray:
    """
    Convert a point's cartesian coordinates to spherical coordinates.

    Parameters
    ----------
    x: float
    y: float
    z: float

    Returns
    -------
    ndarray
        Returns the distance, the vertical angle and the horizontal angle coordinates.
    """
    distance = np.linalg.norm([x, y, z])
    if distance == 0.:
        vertical_angle = 0.
    else:
        vertical_angle = np.arccos(z/distance)
    if x == 0. and y == 0.:
        horizontal_angle = 0.
    else:
        horizontal_angle = np.pi/2. - np.arctan2(y, x)

    return np.array([distance, vertical_angle, horizontal_angle])


def spherical_2_cartesian_coords(distance: float, vertical_angle: float, horizontal_angle: float) -> NDArray:
    """
    Convert a point's spherical coordinates to cartesian coordinates.

    Parameters
    ----------
    distance: float
        The slope distance in meter.
    vertical_angle: float
        The vertical angle in radian. Z-axis reference.
    horizontal_angle: float
        The horizontal angle measured by a total station. Y-axis reference and clockwise.

    Returns
    -------
    ndarray
        Returns the x, y and z coordinates.
    """
    x = distance*np.cos(np.pi/2 - horizontal_angle)*np.sin(vertical_angle)
    y = distance*np.sin(np.pi/2 - horizontal_angle)*np.sin(vertical_angle)
    z = distance*np.cos(vertical_angle)

    return np.array([x, y, z])

def simulate_random_noise_readings(ts1_readings: NDArray, ts2_readings: NDArray, ts3_readings: NDArray, distance_noise: float, angle_noise: float, average_quantity: int, seed: Union[None, int] = 0) -> List[NDArray]:
    """
    Simulate random noise for each reading of the total stations.

    It will simulate average_quantity readings for with noise and return the average for each readings.

    Parameters
    ----------
    ts1_readings: ndarray
        Total station 1 readings in spherical coordinates, i.e. [distance, elevation, azimuth].
    ts2_readings: ndarray
        Total station 2 readings in spherical coordinates, i.e. [distance, elevation, azimuth].
    ts3_readings: ndarray
        Total station 3 readings in spherical coordinates, i.e. [distance, elevation, azimuth].
    distance_noise: float
        Noise to be applied to the distance readings.
    angle_noise: float
        Noise to be applied to the elevation (vertical angle) and to the azimuth (horizontal angle) readings
    average_quantity: int
        The number of reading to simulate to compute the average.
    seed: None or int
        The seed to pass to the random number generator.
    Returns
    -------
    A list of readings of the pillars for each total station where each reading is the average over readings_quantity simulated reading with noise.
    """
    rng = np.random.default_rng(seed)
    normal_distribution = rng.normal
    means = [0, 0, 0]
    stds = [distance_noise, angle_noise, angle_noise]
    ts1_noisy_tmp = [[] for _ in range(len(ts1_readings))]
    ts2_noisy_tmp = [[] for _ in range(len(ts2_readings))]
    ts3_noisy_tmp = [[] for _ in range(len(ts3_readings))]

    for _ in range(average_quantity):
        for i, l in enumerate(ts1_noisy_tmp):
            l.append(ts1_readings[i] + normal_distribution(loc=means, scale=stds))

        for i, l in enumerate(ts2_noisy_tmp):
            l.append(ts2_readings[i] + normal_distribution(loc=means, scale=stds))

        for i, l in enumerate(ts3_noisy_tmp):
            l.append(ts3_readings[i] + normal_distribution(loc=means, scale=stds))

    ts1_noisy = np.array([np.mean(l, axis=0) for l in ts1_noisy_tmp])
    ts2_noisy = np.array([np.mean(l, axis=0) for l in ts2_noisy_tmp])
    ts3_noisy = np.array([np.mean(l, axis=0) for l in ts3_noisy_tmp])

    return [ts1_noisy, ts2_noisy, ts3_noisy]

In [8]:
%matplotlib inline

tfunctions = importlib.reload(tfunctions)
tutils = importlib.reload(tutils)
resection = importlib.reload(resection)

# Real pillars distances and elevations differences
# AB_distance, AC_distance, AD_distance = 181.016, 363.492, 548.578
# AB_elevation_diff, AC_elevation_diff, AD_elevation_diff = 0.323, 1.038, 2.374

# Fake pillars distances and elevations differences
AB_distance, AC_distance, AD_distance = 150, 300, 450
AB_elevation_diff, AC_elevation_diff, AD_elevation_diff = 1, 3, 5

# Convert into cartesian coordinates
A = np.array([0., 0., 0.])
B = np.array([0., np.sqrt(AB_distance**2 - AB_elevation_diff**2), AB_elevation_diff])
C = np.array([0., np.sqrt(AC_distance**2 - AC_elevation_diff**2), AC_elevation_diff])
D = np.array([0., np.sqrt(AD_distance**2 - AD_elevation_diff**2), AD_elevation_diff])

pillars = np.array([A, B, C, D])

# Reference positions of each total stations
TS1_ref_position = np.array([-20., 200., -0.1])
TS2_ref_position = np.array([-20., 275., -0.2])
TS3_ref_position = np.array([-20., 350., -0.3])

# Reference orientation of each total stations
TS1_ref_orientation = resection.R_z(np.pi + np.pi/6)[:3, :3]
TS2_ref_orientation = resection.R_z(np.pi + np.pi/6)[:3, :3]
TS3_ref_orientation = resection.R_z(np.pi + np.pi/6)[:3, :3]

# Pillars readings cartesian coordinates in each total station frame
TS1_ref_readings = np.array([A - TS1_ref_position, B - TS1_ref_position, C - TS1_ref_position, D - TS1_ref_position]).T
TS2_ref_readings = np.array([A - TS2_ref_position, B - TS2_ref_position, C - TS2_ref_position, D - TS2_ref_position]).T
TS3_ref_readings = np.array([A - TS3_ref_position, B - TS3_ref_position, C - TS3_ref_position, D - TS3_ref_position]).T

TS1_ref_readings = (TS1_ref_orientation@TS1_ref_readings).T
TS2_ref_readings = (TS2_ref_orientation@TS2_ref_readings).T
TS3_ref_readings = (TS3_ref_orientation@TS3_ref_readings).T

# Convert reference cartesian coordinates to spherical coordinates for the two points resection function
TS1_ref_readings = np.array([cartesian_2_spherical_coords(*readings) for readings in TS1_ref_readings])
TS2_ref_readings = np.array([cartesian_2_spherical_coords(*readings) for readings in TS2_ref_readings])
TS3_ref_readings = np.array([cartesian_2_spherical_coords(*readings) for readings in TS3_ref_readings])

number_data = 2 # Number of data to simulate
number_noisy_readings = 10 # Number of noisy reading used to average
max_distance_noise = 0.001*(number_data - 1)
distance_noises = np.linspace(0, max_distance_noise, number_data)
max_angle_noise = np.radians((number_data - 1)/3600)
angle_noises = np.linspace(0, max_angle_noise, number_data)
all_control_points_errors = []
all_validation_points_errors = []
all_optimization_errors = []

for distance_noise, angle_noise in zip(distance_noises, angle_noises):
    # Total stations readings with noise in spherical coordinates
    TS1_noisy_readings, TS2_noisy_readings, TS3_noisy_readings = simulate_random_noise_readings(TS1_ref_readings,
                                                                                                TS2_ref_readings,
                                                                                                TS3_ref_readings,
                                                                                                distance_noise,
                                                                                                angle_noise,
                                                                                                number_noisy_readings)
    TS1_all_TFs, TS2_all_TFs, TS3_all_TFs = [[], []], [[], []], [[], []]

    # Compute all total stations possible rigid transformation for the 6 different pillar combinations. i.e. (A, B), (A, C), (A, D), (B, C), (B, D), (C, D)
    pillars_combinations = combinations([i for i in range(len(pillars))], 2)
    for first, second in pillars_combinations:
        TS1_possible_TFs = resection_with_2_known_points(pillars[first],
                                                         pillars[second],
                                                         TS1_noisy_readings[first],
                                                         TS1_noisy_readings[second])
        for pose, station in zip(TS1_possible_TFs, TS1_all_TFs):
            station.append(pose)

        TS2_possible_TFs = resection_with_2_known_points(pillars[first],
                                                         pillars[second],
                                                         TS2_noisy_readings[first],
                                                         TS2_noisy_readings[second])
        for pose, station in zip(TS2_possible_TFs, TS2_all_TFs):
            station.append(pose)

        TS3_possible_TFs = resection_with_2_known_points(pillars[first],
                                                         pillars[second],
                                                         TS3_noisy_readings[first],
                                                         TS3_noisy_readings[second])
        for pose, station in zip(TS3_possible_TFs, TS3_all_TFs):
            station.append(pose)

    # Convert total stations readings with noise in homogeneous coordinates
    TS1_readings_homogeneous = np.array([np.append(spherical_2_cartesian_coords(*reading), 1) for reading in TS1_noisy_readings]).T
    TS2_readings_homogeneous = np.array([np.append(spherical_2_cartesian_coords(*reading), 1) for reading in TS2_noisy_readings]).T
    TS3_readings_homogeneous = np.array([np.append(spherical_2_cartesian_coords(*reading), 1) for reading in TS3_noisy_readings]).T

    # We always use the second rigid transformation found as it is always the correct one.
    TS1_TFs = np.array(TS1_all_TFs[1])
    TS2_TFs = np.array(TS2_all_TFs[1])
    TS3_TFs = np.array(TS3_all_TFs[1])

    # Apply the rigid transformations to realign all the total stations readings in the pillars' frame.
    P1 = TS1_TFs@TS1_readings_homogeneous
    P2 = TS2_TFs@TS2_readings_homogeneous
    P3 = TS3_TFs@TS3_readings_homogeneous

    # Compute the error between the measures of each total station for each pillar's combination
    control_points_errors = []
    validation_points_errors = []

    pillars_combinations = combinations([i for i in range(len(pillars))], 2)
    for (first, second), pillars_1, pillars_2, pillars_3 in zip(pillars_combinations, P1, P2, P3):
        for index, (pillar_1, pillar_2, pillar_3) in enumerate(zip(pillars_1.T, pillars_2.T, pillars_3.T)):
            error_12, error_13, error_23 = resection.compute_error_between_three_points(pillar_1, pillar_2, pillar_3)
            if index == first or index == second:
                control_points_errors.append(error_12)
                control_points_errors.append(error_13)
                control_points_errors.append(error_23)
            else:
                validation_points_errors.append(error_12)
                validation_points_errors.append(error_13)
                validation_points_errors.append(error_23)

    # Compute the error for the optimized resection with 2 know points
    pillars_homogeneous = np.append(pillars.T, [[1.,1.,1.,1.]], axis=0)
    TS1_TF_opt, TS2_TF_opt, TS3_TF_opt, P1_opt, P2_opt, P3_opt, optimization_errors, _, _, _ = resection.geomatic_resection_optimization_on_pose(TS1_readings_homogeneous,
                                                                                                                                                 TS2_readings_homogeneous,
                                                                                                                                                 TS3_readings_homogeneous,
                                                                                                                                                 pillars_homogeneous)

    all_control_points_errors.append(np.mean(control_points_errors))
    all_validation_points_errors.append(np.mean(validation_points_errors))
    all_optimization_errors.append(np.mean(optimization_errors))

In [ ]:
print("Mean error for the control points: ", round(np.mean(all_control_points_errors)*1000, 3), "mm")
print("Median error for the control points: ", round(np.median(all_control_points_errors)*1000, 3), "mm")
print("Mean error for the validation points: ", round(np.mean(all_validation_points_errors)*1000, 3), "mm")
print("Median error for the validation points: ", round(np.median(all_validation_points_errors)*1000, 3), "mm")
print("Mean error for the optimization points: ", round(np.mean(all_optimization_errors)*1000, 3), "mm")
print("Median error for the optimization points: ", round(np.median(all_optimization_errors)*1000, 3), "mm")

In [ ]:
%matplotlib notebook
# Plot the results of the resection with 2 known points (not optimized version)
for p1, p2, p3, tf1, tf2, tf3 in zip(P1, P2, P3, TS1_TFs, TS2_TFs, TS3_TFs):
    tfunctions.plot_trajectories_prism(3, p1, p2, p3, tf1, tf2, tf3, 0, 0, "example.pdf", 0)

In [ ]:
%matplotlib notebook
# Plot the result of the resection with 2 known points (optimized version)
tfunctions.plot_trajectories_prism(3, P1_opt, P2_opt, P3_opt, TS1_TF_opt, TS2_TF_opt, TS3_TF_opt, 0, 0, "example.pdf", 0)

In [ ]:
%matplotlib notebook

plt.plot(np.array(distance_noises), np.array(all_control_points_errors), c="g", label="control points")
plt.plot(np.array(distance_noises), np.array(all_validation_points_errors), c="r", label="validation points")
plt.plot(np.array(distance_noises), np.array(all_optimization_errors), c="b", label="optimized points")
plt.legend(loc="best")
plt.tight_layout()
plt.grid(True)

plt.show()
